In [35]:
import sys
import pandas as pd
import os
import sys
os.makedirs('configs', exist_ok=True)
#!{sys.executable} -m install keplergl
#from keplergl import KeplerGl

import pickle
#!{sys.executable} -m install h3
from h3 import h3

def load_pickle(file_path):
    with open(file_path, 'rb') as handle:
        return pickle.load(handle)
        
def save_pickle(obj, filepath): 
    with open(filepath, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

Для корректной работы KeplerGl в jupyter notebook: https://docs.kepler.gl/docs/keplergl-jupyter

In [36]:
os.makedirs('configs', exist_ok=True)

In [37]:
df_roads = pd.read_csv("train/roads_dataset.csv")
df_transport = pd.read_csv("train/routes_dataset.csv")
df_population = pd.read_csv("train/rosstat_population_all_cities.csv")
df_isochrones = pd.read_csv("train/isochrones_walk_dataset.csv")
df_stops = pd.read_csv("train/osm_stops.csv")
df_companies = pd.read_csv("train/osm_amenity.csv")

## Дороги
**highway** - тип дороги

**geometry** - shapely геометрия дороги

**oneway** - признак одностороннего движения

**name** - название улицы

**city** - город

In [38]:
df_roads.head()

,osmid,highway,geometry,oneway,name,city
0,33997807,primary_link,"LINESTRING (30.3314215 59.895094, 30.3311238 5...",False,NaN,Санкт-Петербург
1,201380605,primary,"LINESTRING (30.3314215 59.895094, 30.3318177 5...",True,Лиговский проспект,Санкт-Петербург
2,33997784,primary_link,"LINESTRING (30.3375246 59.8992885, 30.3379026 ...",False,NaN,Санкт-Петербург
3,178581714,primary,"LINESTRING (30.3375246 59.8992885, 30.3372282 ...",True,Лиговский проспект,Санкт-Петербург
4,4459876,primary,"LINESTRING (30.3395524 59.9017314, 30.3398299 ...",True,Воздухоплавательная улица,Санкт-Петербург


## Маршруты
**geometry** - shapely геометрия маршрута

**type** - тип общественного транспорта

**city** - город

**name** - идентификатор маршрута

In [39]:
df_transport.head()

,geometry,type,city,name
0,"LINESTRING (30.367515 60.012027, 30.367518 60....",tram,Санкт-Петербург,d645920e395fedad7bbbed0eca3fe2e0
1,"LINESTRING (30.231228 60.023206, 30.233159 60....",tram,Санкт-Петербург,b53b3a3d6ab90ce0268229151c9bde11
2,"LINESTRING (30.355657 59.956911, 30.355829 59....",tram,Санкт-Петербург,eccbc87e4b5ce2fe28308fd9f2a7baf3
3,"LINESTRING (30.235013 59.956634, 30.234953 59....",tram,Санкт-Петербург,1679091c5a880faf6fb5e6087eb1b2dc
4,"LINESTRING (30.355545 59.956849, 30.355398 59....",tram,Санкт-Петербург,37693cfc748049e45d87b8c7d8b9aacd


## Население
**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**population** - насление в хексагоне

**lat, lon** - широта/долгота центра хексагоны

In [40]:
df_population.head()

,geo_h3_10,city,population,lat,lon
0,8a0bb0200907fff,Новосибирск,12,54.942110,83.146356
1,8a0bb0200927fff,Новосибирск,8,54.942144,83.144157
2,8a0bb020092ffff,Новосибирск,28,54.943220,83.145270
3,8a0bb0200937fff,Новосибирск,12,54.941034,83.145243
4,8a0bb0200977fff,Новосибирск,10,54.944296,83.146384


## Изохроны 

https://medium.com/@urbica/%D0%B3%D0%B0%D0%BB%D1%8C%D1%82%D0%BE%D0%BD-c7360acf1ec4

**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**walk_15min** - shapely геометрия 15 минутной пешей доступности из точки (lat, lon)

**lat, lon** - широта/долгота центра хексагоны

### Магазины-кампании и прочее 
**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**city** - город

**lat, lon** - широта/долгота центра хексагоны

**остальные колонки** - число организаций типа "название колонки" в хексагоне

In [41]:
df_companies.head()

,geo_h3_10,city,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,Женская одежда,...,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat,lon
0,8a0bb0200a5ffff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,54.953869,83.163008
1,8a0bb0200b37fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,54.948563,83.153038
2,8a0bb0201cd7fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,54.958064,83.174064
3,8a0bb0201d97fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,54.953799,83.167407
4,8a0bb0204027fff,Новосибирск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.935864,83.126481


## Остановки
**osm_id** - идентификатор 

**geo_h3_10** - https://eng.uber.com/h3/ : хексагона - полигон в виде соты

**lat, lon** - широта/долгота центра хексагоны

**type** - тип остановки

**city** - город

**name** - название остановки

In [43]:
#print(len(df_stops))
#print(len(df_stops['osm_id'].unique()))
df_stops.head()
df_stops_res = df_stops.groupby('geo_h3_10').count()
df_stops_res.reset_index(inplace=True)
df_stops_res.rename(columns={'city':'stops_count'},inplace=True)
df_stops_res = df_stops_res[['geo_h3_10','stops_count']]
df_stops_res
df_stops_res.describe().T

,count,mean,std,min,25%,50%,75%,max
stops_count,8788.0,1.401457,0.708085,1.0,1.0,1.0,2.0,8.0


### Визуализируем данные

In [44]:
map_config_path = "configs/map-config.pkl"
if os.path.isfile(map_config_path):
    config = load_pickle(map_config_path)
else:
    config = {}

In [45]:
# map_kepler = KeplerGl(height = 750, config=config)
# map_kepler.add_data(df_roads, 'дороги')
# map_kepler.add_data(df_transport, 'маршруты транспорта')
# map_kepler.add_data(df_population.drop('geo_h3_10', axis = 1), 'Население')
# map_kepler.add_data(df_isochrones.drop('geo_h3_10', axis = 1), 'Пешая доступность')
# map_kepler.add_data(df_stops.drop('geo_h3_10', axis = 1), 'Остановки')
# map_kepler

In [46]:
#save_pickle(map_kepler.config, "configs/map-config.pkl")

## 1. Население  

In [47]:
print(len(df_population))
#Why???

print(len(df_population['geo_h3_10'].unique()))
print(len(df_population.drop_duplicates()))
df_population_by_city = df_population.drop_duplicates().groupby('city').sum()['population']
df_population_by_city

34855
33724
33724


city
Екатеринбург       1787172
Нижний Новгород    1465501
Новосибирск        2002930
Санкт-Петербург    5158815
Name: population, dtype: int64

###  Плотность жилой постройки

In [48]:
#print(df_companies.head())
feature_cols = list(df_companies)
feature_cols.remove('city')
feature_cols.remove('lat')
feature_cols.remove('lon')
print('Feature cols:', feature_cols)
df_companies.fillna(0,inplace=True)
print(len(df_companies.drop_duplicates()))
df_companies_res = df_companies.drop_duplicates(subset=['geo_h3_10'])
df_companies_res
#df_total_bulding_by_city = df_companies.drop_duplicates().groupby('city')[feature_cols].agg('sum')
#df_total_bulding_by_city

Feature cols: ['geo_h3_10', 'Автозапчасти для иномарок', 'Авторемонт и техобслуживание (СТО)', 'Алкогольные напитки', 'Аптеки', 'Банки', 'Быстрое питание', 'Доставка готовых блюд', 'Женская одежда', 'Кафе', 'Косметика / Парфюмерия', 'Ногтевые студии', 'Овощи / Фрукты', 'Парикмахерские', 'Платёжные терминалы', 'Постаматы', 'Продуктовые магазины', 'Пункты выдачи интернет-заказов', 'Рестораны', 'Страхование', 'Супермаркеты', 'Цветы', 'Шиномонтаж']
24599


,geo_h3_10,city,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,Женская одежда,...,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat,lon
0,8a0bb0200a5ffff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,54.953869,83.163008
1,8a0bb0200b37fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,54.948563,83.153038
2,8a0bb0201cd7fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,54.958064,83.174064
3,8a0bb0201d97fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,54.953799,83.167407
4,8a0bb0204027fff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.935864,83.126481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24594,8a1106c49667fff,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,59.708838,30.021463
24595,8a1106cc8367fff,Санкт-Петербург,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.571796,30.123930
24596,8a1106cc9407fff,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,59.574353,30.139236
24597,8a11154206effff,Нижний Новгород,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.524786,31.234655


## 3. Проходимость локации  

In [49]:
def convert_to_poly(string):
    return shapely.wkt.loads(string)
def get_area(polygon):
    return polygon.area

In [50]:
df_isochrones.head()
#len(df_isochrones['walk_15min'][13])
import shapely.wkt
walking_ploygones = df_isochrones['walk_15min']
Polygones = walking_ploygones.apply(convert_to_poly)
df_isochrones['area'] = Polygones.apply(get_area)
df_isochrones['area'][:10]
df_with_areas = df_isochrones.copy()
df_with_areas

,city,walk_15min,geo_h3_10,lat,lon,area
0,Нижний Новгород,"POLYGON ((43.776762 56.391904, 43.775762 56.39...",8a101601044ffff,56.383627,43.777762,0.000259
1,Нижний Новгород,"POLYGON ((43.773717 56.39169, 43.772717 56.390...",8a101601045ffff,56.382411,43.777717,0.000237
2,Нижний Новгород,"POLYGON ((43.773458 56.388816, 43.770682 56.38...",8a10160104cffff,56.380606,43.779458,0.000228
3,Нижний Новгород,"POLYGON ((43.788412 56.382862, 43.784155 56.38...",8a10160104dffff,56.379391,43.779412,0.000158
4,Нижний Новгород,"POLYGON ((43.775051 56.391224, 43.773051 56.39...",8a1016010607fff,56.384913,43.785051,0.000268
...,...,...,...,...,...,...
47092,Екатеринбург,"POLYGON ((60.595314 56.77692, 60.594258 56.776...",8a10dc936d97fff,56.767846,60.585314,0.000345
47093,Екатеринбург,"POLYGON ((60.596524 56.777762, 60.594996 56.77...",8a10dc936d9ffff,56.768217,60.587524,0.000382
47094,Екатеринбург,"POLYGON ((60.596222 56.777785, 60.595956 56.77...",8a10dc936da7fff,56.769914,60.584222,0.000343
47095,Екатеринбург,"POLYGON ((60.597432 56.778491, 60.596432 56.77...",8a10dc936daffff,56.770285,60.586432,0.000383


### Сортировка по площади и отображение:  

In [51]:
df = df_with_areas[['geo_h3_10','city','area']]
cities = ['Нижний Новгород','Новосибирск','Екатеринбург','Санкт-Петербург']

In [64]:
df_array = []
for elem in cities:
    print(elem)
    df_city_by_area = df[df['city']==elem]
    df_city_by_area['score'] = pd.cut(df_city_by_area['area'], 10, labels=range(1,11))
    df_array.append(df_city_by_area.sort_values(by=['score','area'],ascending=False))
#df_array[0]
df_city_by_area_res = pd.concat(df_array)
df_city_by_area_res

Нижний Новгород
Новосибирск
Екатеринбург
Санкт-Петербург


C:\Users\dyush\AppData\Local\Temp/ipykernel_25144/3447138461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city_by_area['score'] = pd.cut(df_city_by_area['area'], 10, labels=range(1,11))
C:\Users\dyush\AppData\Local\Temp/ipykernel_25144/3447138461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city_by_area['score'] = pd.cut(df_city_by_area['area'], 10, labels=range(1,11))
C:\Users\dyush\AppData\Local\Temp/ipykernel_25144/3447138461.py:5: SettingWithCopyWarning: 
A value is trying to be set 

,geo_h3_10,city,area,score
8039,8a1016735c27fff,Нижний Новгород,0.000604,10
8060,8a1016735d17fff,Нижний Новгород,0.000590,10
8041,8a1016735c37fff,Нижний Новгород,0.000584,10
8059,8a1016735d0ffff,Нижний Новгород,0.000582,10
8061,8a1016735d1ffff,Нижний Новгород,0.000580,10
...,...,...,...,...
17616,8a11060ec487fff,Санкт-Петербург,0.000021,1
17618,8a11060ec49ffff,Санкт-Петербург,0.000020,1
21372,8a110615c95ffff,Санкт-Петербург,0.000008,1
21369,8a110615c91ffff,Санкт-Петербург,0.000007,1


### Read target:  

In [53]:
target_data = pd.read_csv('target_hakaton_spb.csv',sep=';',encoding='windows-1251')

In [54]:
target_data

,gorod,geo_h3_10,lat_h3,lon_h3,atm_category,target,atm_cnt
0,Екатеринбург,8a10c2283357fff,56.865861,60.532224,category4,95.333333,1
1,Екатеринбург,8a10c2288087fff,56.885391,60.588203,category3,30652.666667,4
2,Екатеринбург,8a10c228820ffff,56.891648,60.605399,category2,1162.166667,1
3,Екатеринбург,8a10c2288adffff,56.895039,60.598777,category1,2961.333333,1
4,Екатеринбург,8a10c228944ffff,56.896206,60.625911,category1,2963.333333,1
...,...,...,...,...,...,...,...
533,Санкт-Петербург,8a1106ab67a7fff,59.983488,30.227538,category1,2535.833333,1
534,Санкт-Петербург,8a1106ab6807fff,59.990519,30.205250,category1,23519.500000,2
535,Санкт-Петербург,8a1106ab6807fff,59.990519,30.205250,category4,94.500000,1
536,Санкт-Петербург,8a1106ab6ae7fff,59.996443,30.219656,category2,489.500000,1


In [55]:
target_data.describe().T
#means = target_data

,count,mean,std,min,25%,50%,75%,max
lat_h3,538.0,57.840711,2.100293,54.839007,56.283908,56.894509,59.931663,60.066932
lon_h3,538.0,49.504859,21.473169,30.093182,30.327388,43.885253,60.646317,83.123223
target,538.0,7982.345415,10846.008692,0.000000,261.666667,2958.416667,12044.541667,82499.166667
atm_cnt,538.0,1.315985,0.755221,1.000000,1.000000,1.000000,1.000000,6.000000


In [56]:
tr_df = target_data[target_data['gorod'] == 'Санкт-Петербург']
tr_df['atm_cnt'].sum()

337

In [57]:
df_companies_stops = df_companies_res.merge(df_stops_res,on='geo_h3_10')
df_companies_stops

,geo_h3_10,city,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,Женская одежда,...,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat,lon,stops_count
0,8a0bb0204067fff,Новосибирск,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,3.0,0.0,0.0,54.939126,83.127621,2
1,8a0bb02041affff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.933678,83.126455,1
2,8a0bb0204327fff,Новосибирск,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.942388,83.128762,2
3,8a0bb020496ffff,Новосибирск,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,54.939436,83.107828,1
4,8a0bb0204aaffff,Новосибирск,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,54.942458,83.124363,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,8a1106ba205ffff,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,2.0,0.0,0.0,60.040220,29.978040,2
4055,8a1106ba398ffff,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,60.057065,29.977840,1
4056,8a1106ba3d17fff,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.049922,29.976602,2
4057,8a1106ba3d1ffff,Санкт-Петербург,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,2.0,0.0,0.0,60.050707,29.978156,2


In [68]:
df_final = df_city_by_area_res.merge(df_companies_stops,on='geo_h3_10')
df_final

,geo_h3_10,city_x,area,score,city_y,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,...,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,lat,lon,stops_count
0,8a1016735d1ffff,Нижний Новгород,0.000580,10,Нижний Новгород,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.259172,44.004244,2
1,8a101608b487fff,Нижний Новгород,0.000558,10,Нижний Новгород,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.315593,43.986771,2
2,8a1016724857fff,Нижний Новгород,0.000554,10,Нижний Новгород,0.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,1.0,0.0,2.0,0.0,56.311350,43.988399,1
3,8a101672495ffff,Нижний Новгород,0.000547,10,Нижний Новгород,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.0,56.312537,43.984831,1
4,8a101672425ffff,Нижний Новгород,0.000545,10,Нижний Новгород,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,56.312170,44.015580,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050,8a110604e19ffff,Санкт-Петербург,0.000190,4,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,59.935603,30.501015,1
3051,8a110615da07fff,Санкт-Петербург,0.000189,4,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,59.923266,30.252681,2
3052,8a1106a9b737fff,Санкт-Петербург,0.000166,3,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,60.026271,30.453735,1
3053,8a1106001a6ffff,Санкт-Петербург,0.000164,3,Санкт-Петербург,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.818240,30.328448,1


In [ ]:
df_final.to_csv("result_df.csv")